# WFI Reference File Pipeline - ReadNoise Example

This notebook shows how to use the WFI Reference File Pipeline (RFP) to create readnoise reference files. The RFP will simulate dark reads that will be used as input into the ReadNoise module. 

Additional functionality such as reading a list of files to construct the input data, and optional methods to calculate the read noise are in development and only mentioned, but are not necessary to generate reference files suitable for delivery to CRDS.

## Imports

The imports here are used in the below cells.  
NOTE: We are configuring logging which will require you to have a config.yml file in the /src/wfi_reference_pipeline/config directory.

In [1]:
import os
import logging
from wfi_reference_pipeline.utilities import simulate_reads
from wfi_reference_pipeline.resources.make_dev_meta import MakeDevMeta
from wfi_reference_pipeline.readnoise import readnoise
from wfi_reference_pipeline.constants import WFI_MODE_WIM, WFI_FRAME_TIME
from wfi_reference_pipeline.utilities import logging_functions
logging_functions.configure_logging("readnoise_example")

'/Users/bsappington/tmp/logs/BRAD_DEV_readnoise_example_bsappington_2023-11-30-17-03.log'

## Setup 

The example read noise reference file will be created in the RFP scratch directory. This directory has it contents automatically deleted every night. A default outfile name is given and if clobber set to True will be overwritten.

In [2]:
write_path = '/grp/roman/RFP/DEV/scratch/'  # Set the write path to be in the RFP scratch directory.
outfile = write_path + 'roman_dev_readnoise.asdf'  # Default read noise development filename.

## Meta Data

The RFP utilizes python data classes to construct reference file type dependent meta data for development from the "ref_type" keyword, which is then exported into a dictionary for valid usage with roman data models. 

In [3]:
logging.info("Generating Readnoise Metadata")
tmp = MakeDevMeta(ref_type='READNOISE')  # Get the default reference file type specific development meta data.
readnoise_dev_meta = tmp.meta_readnoise.export_asdf_meta()  # Export the meta data as a dictionary.

In [4]:
print(readnoise_dev_meta)  # Inspect default development meta data.

{'reftype': 'READNOISE', 'pedigree': 'DUMMY', 'description': 'For RFP Development and DMS Build Support.', 'author': 'RFP', 'useafter': <Time object: scale='utc' format='isot' value=2023-01-01T00:00:00.000>, 'telescope': 'ROMAN', 'origin': 'STSCI', 'instrument': {'name': 'WFI', 'detector': 'WFI01'}, 'exposure': {'type': 'WFI_IMAGE', 'p_exptype': 'WFI_IMAGE|'}}


In [5]:
readnoise_dev_meta.update({'useafter': '2020-01-01T00:00:00.000'})  # Update to the useafter date found on CRDS to replace an existing file.
# Or make a new useafter date.

det = 5  # Make the readnoise reference file for WFI05.
readnoise_dev_meta['instrument'].update({'detector': 'WFI' + f"{det:02d}"})

In [6]:
print(readnoise_dev_meta)  # Inspect updated meta data.

{'reftype': 'READNOISE', 'pedigree': 'DUMMY', 'description': 'For RFP Development and DMS Build Support.', 'author': 'RFP', 'useafter': '2020-01-01T00:00:00.000', 'telescope': 'ROMAN', 'origin': 'STSCI', 'instrument': {'name': 'WFI', 'detector': 'WFI05'}, 'exposure': {'type': 'WFI_IMAGE', 'p_exptype': 'WFI_IMAGE|'}}


## Simulate Dark Read Data

Use the RFP simulate reads function to create a cube of dark reads with input parameters. The current assumed input to create the read noise reference file is a series of dark reads. The read noise reference file is WFI mode specific and therefore CRDS requires that there are separate WFI Imaging Mode (WIM) or WFI Spetral Mode (WSM) reference files available for the WFI optical element filters and WFI spectral components GRISM and PRISM.

In [7]:
tot_reads = 10  # The number of reads in the simulated cube.
dark_rate = 0.005  # The rate in e/p/s electrons per pixel per second
dark_var = 0.001  # The variance in the dark rate.

# We are using the WFI_MODE_WIM which has a frame time of 3.04 seconds for each read to simulate the data cube.
sim_read_cube, _ = simulate_reads.simulate_reads(tot_reads, WFI_FRAME_TIME[WFI_MODE_WIM], dark_rate, dark_var)

Simulating WFI dark reads...


## Instantiate RFP readnoise

The reference file pipeline has a reference file base class which when any reference file type, like ReadNoise here, is instantiated it inherits the base class attributes and methods. The reference file base class has positional arguments and it is necessary to declare the data, the first positional input to the reference file base class, as None to properly inherit the reference file base class attributes. If None is replaced by a file or list of files, ReadNoise will follow a different flow to create input data to generate a reference file. 

In [8]:
RFPreadnoise = readnoise.ReadNoise(None,
                                   meta_data=readnoise_dev_meta,
                                   outfile=outfile,
                                   clobber=True,
                                   input_data_cube=sim_read_cube)

## Compute Readnoise

The readnoise is determined by fitting a linear ramp to the input data cube. The variance of the residuals from the ramp is set by the ReadNoise module to be the readnoise "data" in the asdf file datamodel. 

In [9]:
RFPreadnoise.comp_ramp_res_var()  # Compute the variance of the residuals in the fitted ramp.

## Other ReadNoise Calculations

Other methods are available for use, such as computing the correlated-double-sampling (CDS) noise. Such methods could be used to compare the implemented algorithm in the RFP and its performance as compared to another method. See RFP documentation.

## Build the datamodel for the reference file, write it to disk, and change the permissions of the file. 

Other methods are available to determine the readnoise in line with what others have called the "total noise" or correlated-double-sampling noise (CDS noise). See RFP documentation.

In [11]:
RFPreadnoise.save_readnoise()  # Save the readnoise object into the data model
os.chmod(outfile, 0o666)  # Set file permissions to read+write for owner, group, and global.
print('Made file -> ', outfile)
logging.info(f'outfile created: {outfile}')

Made file ->  /grp/roman/RFP/DEV/scratch/roman_dev_readnoise.asdf
